In [1]:
import cv2
import numpy as np
from math import sqrt

def draw_guidelines(frame):
    height, width = frame.shape[:2]
    #in general camera cover area up to 2.5 times length of car feets extending backward from cars rare bumper and width considering
    #car width about 5.8ft exteding 1.5 feet on both sides 8.8~9 feet 
    
    #real word coverage area
    coverage_width_feet=12
    coverage_depth_feet=50
    
    width_conversion_factor=width/coverage_width_feet
    depth_conversion_factor=height/coverage_depth_feet
    
    #Now to draw the fixed green lines as per the vehicle width considering vehicle as 5 feet for toyata prius
    # Static guidelines for demonstration
    # These would be dynamically calculated in a real application
    vehicle_width=5
    vehicle_length=14
    
    vehicle_depth=(height*vehicle_length)/coverage_depth_feet
    start_width=(width*vehicle_width)/coverage_width_feet #finding number of pixels needed for vehicle width
    left_point=(width/2)-(start_width/2) #left tyre projection start point
    right_point=(width/2)+(start_width/2) #right tyre projection start point
    
    left_line_start = (int(left_point), height)
    left_line_end = (int(left_point + 100), int(height-vehicle_depth))

    right_line_start = (int(right_point), height)
    right_line_end = (int(right_point - 100), int(height-vehicle_depth))

    center_line_start = (int(width / 2), height)
    center_line_end = (int(width / 2), int(height-vehicle_depth))

    # Draw the lines
    cv2.line(frame, left_line_start, left_line_end, (0, 255, 0), 3)
    cv2.line(frame, right_line_start, right_line_end, (0, 255, 0), 3)
    cv2.line(frame, center_line_start, center_line_end, (0, 255, 0), 2)
    
    

    
    

    # For a typical car, the turning circle might be around 35 to 40 feet in diameter, we use half of that for the radius
    turning_circle_diameter_feet = 40
    turning_radius_feet = turning_circle_diameter_feet / 2
    
    
    #turing radius for rare wheel is lower than the front wheel which is given by below equation
    rare_wheel_radius=sqrt(turning_radius_feet**2-vehicle_length**2)
    
    rare_wheel_pixels = int(rare_wheel_radius * width_conversion_factor)
    turning_radius_pixels=int(turning_radius_feet*width_conversion_factor)
    
    left_ellipse_center = (int(left_point +  rare_wheel_pixels-590), int(height+1050))  # This assumes the center of the circle is directly below the image
    right_ellipse_center = (int(right_point - rare_wheel_pixels+590),int(height+1050))  # Same for the right side

    # The axes of the ellipse (the second value is small to create a flatter arc)
    axes_length = ( rare_wheel_pixels,  rare_wheel_pixels) # The second value is arbitrary and should be adjusted

    # Drawing the left arc
    cv2.ellipse(frame,right_ellipse_center , axes_length, 0, 280, 320, (0, 0, 255), 3)
    # Drawing the right arc
    cv2.ellipse(frame,left_ellipse_center , axes_length, 0, 180, 260, (0, 0, 255), 3)
    
    
    
    
    return frame

# Video capture from a file or camera
cap = cv2.VideoCapture('videocamera.mp4')  # Replace with your video file

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break
    
    frame = cv2.resize(frame, (1024,512))
    # Function to draw guidelines
    frame_with_guidelines = draw_guidelines(frame)

    # Display the resulting frame
    cv2.imshow('Backup Camera View', frame_with_guidelines)

    # Press 'q' to exit the loop
    if cv2.waitKey(8) & 0xFF == ord('q'):
        break

# Release the video capture object and close all frames
cap.release()
cv2.destroyAllWindows()
